In [2]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime

In [36]:
class CrawlStock3PRatioTW:
    def __init__(self, date):
        self.date = date
        self.date_str = date.strftime("%Y%m%d")
        self.target_name = "台股3p數據"
        self.sub_market = ["sii", "otc"]

    def crawl_sii(self):
        r= requests.get('http://www.twse.com.tw/exchangeReport/BWIBBU_d?response=csv&date='+self.date_str+'&selectType=ALL')
        content = r.text.replace('=', '')
        lines = content.split('\n')
        lines = list(filter(lambda l:len(l.split('",')) > 5, lines))
        content = "\n".join(lines[:])
        if content == '':
            return None
        df= pd.read_csv(StringIO(content), header=0).dropna(how='all',axis=1)
        df = df.astype(str)
        df = df.apply(lambda s: s.str.replace(',', ''))
        df=df.drop(columns=['財報年/季','股利年度'])
        df.iloc[:,2:]=df.iloc[:,2:].apply(lambda s:pd.to_numeric(s, errors='coerce'))
        df=df.rename(columns={k:v for k,v in zip(df.columns,['stock_id','stock_name','dividend_yield','pe','pb'])})
        df['date']=self.date
        return df


    def crawl_otc(self):
        y = str(int(self.date.strftime("%Y")) - 1911)
        date_str = y + "/" + self.date.strftime("%m") + "/" + self.date.strftime("%d")
        link = 'https://www.tpex.org.tw/web/stock/aftertrading/peratio_analysis/pera_download.php?l=zh-tw&d='+date_str+'&s=0,asc,0'
        r = requests.get(link)
        lines = r.text.replace('\r', '').split('\n')
        if len(lines)<10:
            return None
        df = pd.read_csv(StringIO("\n".join(lines[3:-1])), header=0)
        df = df.astype(str)
        df = df.apply(lambda s: s.str.replace(',', ''))
        df=df.drop(columns=['每股股利','股利年度'])
        df=df.rename(columns={k:v for k,v in zip(df.columns,['stock_id','stock_name','pe','dividend_yield','pb'])})
        df.iloc[:,2:]=df.iloc[:,2:].apply(lambda s:pd.to_numeric(s, errors='coerce'))
        df['date']=self.date
        return df

    def crawl_main(self):
        try:
            df = pd.concat([self.crawl_sii(), self.crawl_otc()])
        except ValueError:
            return None
        return df
    
z=CrawlStock3PRatioTW(datetime.datetime(2019,1,8)).crawl_main()
z

,stock_id,stock_name,dividend_yield,pe,pb,date
0,1101,台泥,7.03,9.77,1.07,2019-01-08
1,1102,亞泥,3.46,10.6,0.84,2019-01-08
2,1103,嘉泥,3.61,4.89,0.43,2019-01-08
3,1104,環泥,5.87,10.71,0.69,2019-01-08
4,1108,幸福,0,NaN,0.69,2019-01-08
...,...,...,...,...,...,...
759,9949,琉園,0,NaN,0.82,2019-01-08
760,9950,萬國通,0,NaN,1.26,2019-01-08
761,9951,皇田,6.81,10.21,2.34,2019-01-08
762,9960,邁達康,0,32.11,2.2,2019-01-08
